In [2]:
import ccxt
import pandas as pd
from IPython.display import display, HTML

# Function to fetch BTC/USDT historical data from Binance
def fetch_data(symbol, timeframe='1w', since='2000-01-01T00:00:00Z'):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=exchange.parse8601(since))

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Extract year, month, and week for pivot table analysis
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week  # Get the week number of the year
    df['week_of_month'] = (df.index.day - 1) // 7 + 1  # Calculate which week of the month
    df['month_week'] = 'M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    # Generate a unique "week number" for sorting (unique within the same year)
    df['month_letter'] = df['month'].apply(lambda x: chr(64 + x))  # Convert month to letter (1 -> A, 2 -> B, ..., 12 -> L)
    df['unique_week_number'] = df['month_letter'] + '-M' + df['month'].astype(str) + '-W' + df['week_of_month'].astype(str)

    return df

# Function to calculate max drawdown (from open to low)
def calculate_max_drawdown(df):
    df['max_drawdown'] = (df['low'] - df['open']) / df['open'] * 100
    return df

# Function to calculate open-to-high percentage change
def calculate_high_to_open_change(df):
    df['high_to_open_change'] = (df['high'] - df['open']) / df['open'] * 100
    return df

# Function to calculate weekly return (from open to close)
def calculate_weekly_return(df):
    df['weekly_return'] = (df['close'] - df['open']) / df['open'] * 100
    return df

# Function to create a heatmap-style pivot table
def create_pivot_table(df, value_column):
    # Sort by the unique_week_number column to ensure correct order
    df = df.sort_values('unique_week_number')
    
    # Create the pivot table
    pivot_table = df.pivot_table(values=value_column, index='year', columns='unique_week_number', aggfunc='mean')

    # Calculate mean and median for each column
    avg_value = pivot_table.mean(axis=0)
    median_value = pivot_table.median(axis=0)

    # Add the average and median as the last row
    pivot_table.loc['Avg'] = avg_value
    pivot_table.loc['Median'] = median_value

    # Apply color gradients for heatmap (Green for positive, Red for negative)
    def color_map(val):
        color = 'white'  # Default color for no value
        if isinstance(val, (int, float)):  # Check if value is numeric
            if val > 0:
                color = f'rgb({255 - int(val*2.5)}, 255, {255 - int(val*2.5)})'  # Green shades for positive
            elif val < 0:
                color = f'rgb(255, {255 + int(val*2.5)}, {255 - int(abs(val)*2.5)})'  # Red shades for negative
        return f'background-color: {color};'

    # Apply heatmap coloring
    styled_table = pivot_table.style \
        .applymap(color_map)  # Apply the heatmap color function to each cell

    # Additional table styling
    styled_table = styled_table.set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', '#4CAF50'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('text-align', 'center'), ('color', 'black')]},
        {'selector': 'tbody tr:nth-child(odd)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tbody tr:nth-child(even)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
        {'selector': 'th', 'props': [('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]},
        {'selector': 'tbody tr', 'props': [('color', 'blue')]},
    ]) \
    .set_table_attributes('class="dataframe"')

    return styled_table

# Function to display the table with a title
def display_table(title, styled_table):
    display(HTML(f"<h3>{title}</h3>"))
    display(styled_table)

# Main function to fetch data, process it, and display the results
def main():
    # Fetch data
    symbol = 'BTC/USDT'
    BTC_data = fetch_data(symbol)

    # Calculate indicators
    BTC_data = calculate_max_drawdown(BTC_data)
    BTC_data = calculate_high_to_open_change(BTC_data)
    BTC_data = calculate_weekly_return(BTC_data)

    # Create and display the pivot tables with heatmap style
    styled_table_max_drawdown = create_pivot_table(BTC_data, 'max_drawdown')
    display_table("BTC/USDT Weekly Max Drawdown Heatmap (Month-Letter-WeekOfMonth)", styled_table_max_drawdown)

    styled_table_high_to_open_change = create_pivot_table(BTC_data, 'high_to_open_change')
    display_table("BTC/USDT Weekly Open to High % Change Heatmap (Month-Letter-WeekOfMonth)", styled_table_high_to_open_change)

    styled_table_weekly_return = create_pivot_table(BTC_data, 'weekly_return')
    display_table("BTC/USDT Weekly Returns Heatmap (Open to Close, Month-Letter-WeekOfMonth)", styled_table_weekly_return)

# Run the main function
main()


C:\Users\nicol\AppData\Local\Temp\ipykernel_21624\960969152.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-9.655800,-16.444056,-4.303238,nan,-20.022198,-32.179641,-4.998645,-0.172950,nan,-6.590909,-1.939655,-11.769702,-11.514979,-1.679599,-27.502553,-2.396429,-3.178683,-6.660824,nan,-3.254784,-1.871371,-41.882414,-12.962963,nan
2018,-7.040498,-29.711416,-32.964732,-14.134952,-33.248877,-26.650155,-0.134180,-10.604424,-2.502607,-27.668259,-23.197711,-1.228477,-24.086261,nan,-4.594298,-5.705858,-6.339504,-1.526116,-6.547174,-15.609325,-8.846033,-15.197192,-4.170056,nan,-14.148473,-9.553880,-10.770539,-5.831752,nan,-1.402335,-9.566306,-0.373771,-0.267486,-16.181971,-14.993757,-6.793309,-3.975744,-0.798507,nan,-16.310600,-1.320802,-5.801159,-5.692572,nan,-2.966391,-6.201721,-0.620569,-2.080690,-3.773692,-2.017469,-16.235267,-35.487483,-9.772568,nan,-22.558316,-11.053434,-0.409387,-10.044253,-4.543375
2019,-13.700403,-0.303086,-2.950052,-5.637385,nan,-2.458279,-3.033902,-0.344092,-1.773446,-3.588614,-2.695615,-2.557605,-2.571561,nan,-0.863876,-5.977444,-3.532842,-2.956018,-2.467320,-2.668200,-1.409825,-8.430843,-7.054133,nan,-14.685079,-1.528272,-0.033508,-3.492130,nan,-10.384094,-11.454864,-10.951055,-13.457316,-1.535811,-0.020037,-15.504529,-5.280041,-8.111162,nan,-0.114053,-4.828325,-6.311934,-22.716779,-4.140723,-1.187733,-5.549446,-11.228392,-6.456918,nan,-5.441877,-7.626856,-20.144610,-5.582857,nan,-4.390178,-6.681127,-9.615709,-5.656665,-7.002706
2020,-0.147874,-1.577037,-5.329061,-0.921597,nan,-2.556692,-5.059241,-5.657522,-15.351636,-6.216663,-52.927903,-17.129729,-2.201666,-0.386702,-0.484292,-6.308419,-5.201224,-1.132183,nan,-8.737766,-5.993165,-10.134272,-0.865093,nan,-1.929136,-6.504470,-4.620481,-4.967245,-2.447631,-0.148742,-2.752354,-0.846890,-0.144288,nan,-1.222614,-4.765492,-4.484766,-4.554211,-16.105735,-3.957628,-1.166185,-7.174358,-3.714965,nan,-1.327693,-1.732955,-0.827426,-2.031061,nan,-4.116124,-4.326693,-0.582816,-12.088055,-1.011163,-9.227597,-0.912595,-6.994254,-1.527840,nan
2021,-14.757705,-20.262165,-19.469622,-9.354561,nan,-2.407792,-2.079612,-6.195247,-25.103223,-0.406743,-3.305474,-9.672973,-12.072906,-1.752745,-4.688859,-15.112802,-16.420318,-0.638559,nan,-6.501107,-24.751433,-35.382192,-3.755438,-4.173309,-13.398895,-14.568115,-19.087465,-5.773332,nan,-9.099745,-9.454450,-7.868730,-0.495294,nan,-6.317573,-2.318472,-6.484719,-6.070811,-4.625671,-17.222503,-5.769075,-16.175840,-5.576853,nan,-2.715788,-1.427047,-3.279287,-4.982924,nan,-3.091054,-1.573453,-15.139262,-9.145804,-26.668912,-5.355298,-9.185898,-2.404371,-10.039255,nan
2022,-14.349182,-5.289956,-21.043210,-9.180360,-4.307483,-1.734320,-9.639235,-10.588537,-0.660255,-3.294570,-0.588580,-1.924696,-5.611543,nan,-9.781552,-7.018337,-2.877153,-5.231264,nan,-12.359251,-21.559157,-8.536594,-7.507676,-0.630309,-11.227573,-33.688385,-4.554146,-11.465305,nan,-1.348738,-9.348352,-0.178513,-8.285508,nan,-3.847640,-2.199411,-14.578538,-9.279769,-0.079824,-7.447538,-11.485201,-6.651079,-1.796202,nan,-0.514542,-6.429849,-3.182166,-2.112374,-2.890513,-25.436175,-3.162974,-4.935655,-2.638664,nan,-2.500899,-3.264433,-2.883685,-2.965225,nan
2023,-0.412044,-0.135277,-2.231784,-1.788160,-5.236704,-6.461936,-1.980042,-6.387926,-6.723541,-12.844936,-0.837211,-4.901428,-5.219921,nan,-3.448937,-0.542866,-10.492314,-2.347793,nan,-5.357778,-9.210769,-2.067085,-3.274627,

C:\Users\nicol\AppData\Local\Temp\ipykernel_21624\960969152.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,5.254278,9.456075,14.598110,nan,6.295006,5.801445,11.739837,20.396063,nan,5.863636,27.635776,8.073555,3.596318,23.760600,5.785081,39.096463,16.046573,29.546450,nan,54.091878,32.465781,2.332763,22.207778,nan
2018,25.230959,0.637838,5.727935,6.192541,0.800256,10.828742,39.809668,13.599987,20.594369,1.693443,3.726935,12.065087,0.528327,nan,10.377058,20.232376,8.137933,11.087563,6.402861,0.296553,2.307450,0.801457,6.100294,nan,0.657743,2.320769,6.171265,6.370857,nan,7.189953,1.340266,21.180646,14.834654,0.755207,1.933461,4.936784,6.252538,9.633582,nan,1.479047,5.321756,5.214551,1.610154,nan,1.062842,1.516796,21.148258,0.741716,0.540376,1.993571,0.733098,0.036383,8.846110,nan,0.443451,1.733413,28.204025,6.827221,5.905415
2019,2.060878,6.971555,3.494516,0.216898,nan,7.965912,0.553298,14.461149,3.858359,4.318786,3.600367,2.196718,3.724304,nan,28.582258,4.868798,3.939368,6.516326,11.115087,30.288333,20.059011,7.266377,5.361020,nan,0.858927,22.358759,27.249414,28.095214,nan,10.557301,15.224189,9.099701,0.878269,15.704593,12.815291,0.336335,6.246646,4.548202,nan,12.158884,0.951331,0.501391,0.218786,5.681434,11.765432,1.543883,26.104325,3.923814,nan,3.449371,0.355746,0.007644,13.764324,nan,4.850166,1.862689,5.603405,2.595354,1.442390
2020,14.914565,12.388683,1.336288,11.186050,nan,8.941777,3.430749,3.423573,0.539431,7.710163,1.799058,28.723418,19.633772,22.404557,10.142216,5.638787,8.658410,22.967334,nan,13.186758,13.943163,2.777471,11.721078,nan,9.861382,2.521086,2.642875,5.221368,1.928882,4.416936,0.435436,9.794885,22.070293,nan,7.574318,3.301406,4.676437,1.517670,2.900479,3.161305,8.196682,0.628006,1.631110,nan,7.728596,3.087249,16.055268,8.214809,nan,15.975814,6.493658,18.856301,5.812626,9.364861,0.321510,26.701500,21.173932,32.329042,nan
2021,27.121020,5.111347,5.652507,19.443760,nan,23.758943,28.126166,20.115758,0.167420,16.630194,21.360047,2.810207,1.881675,7.928534,5.666454,8.092162,2.452003,19.139862,nan,5.164161,2.162006,0.558233,17.760393,10.759272,10.011339,5.918521,0.420869,5.467936,nan,0.016011,1.224291,11.389531,20.400712,nan,13.700610,9.931964,5.925045,2.560520,6.422593,2.247276,6.122664,0.223319,14.057121,nan,17.347092,15.137468,8.892946,4.697298,nan,4.845349,9.050254,1.347256,1.409642,3.321002,5.142104,0.271847,10.986769,2.584948,nan
2022,0.600218,6.295005,1.006091,7.381606,12.603034,8.117176,6.415020,5.109974,20.427342,10.861981,12.236565,13.903530,2.908702,nan,1.708005,0.606895,8.311611,3.414920,nan,4.043376,0.601556,0.000032,1.243087,9.946009,6.171422,1.209090,6.386702,2.385056,nan,16.626466,3.947201,16.717453,9.261009,nan,1.508047,8.082931,3.727878,1.781407,5.219167,9.298361,4.453822,2.773536,8.382791,nan,7.436664,2.633287,2.303278,7.739239,4.136084,0.785388,5.254908,3.274855,5.449160,nan,1.856588,7.625965,1.925264,0.836021,nan
2023,3.368925,24.113796,11.971500,5.525055,2.154833,2.263516,15.919434,4.027148,1.889844,0.766599,29.057348,3.200172,4.350071,nan,2.140894,9.448746,0.595090,8.863203,nan,2.007307,0.706716,2.771346,5.658563,1.361589,1.019981,3.527217,19.331678,2.689620,nan,2.883951,5.449109,0.613456,0.052620,2.617423,3.972610,1.335900,2.402711,7.819658,nan,1.826214,4.049285,3.604071,6.922787,nan,2.098485,0.253859,11.879772,17.629564,4.226134,8.455270,2.471041,2.821612,7.484040,nan,11.827552,0.034255,7.307906,

C:\Users\nicol\AppData\Local\Temp\ipykernel_21624\960969152.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_map)  # Apply the heatmap color function to each cell


unique_week_number,A-M1-W1,A-M1-W2,A-M1-W3,A-M1-W4,A-M1-W5,B-M2-W1,B-M2-W2,B-M2-W3,B-M2-W4,C-M3-W1,C-M3-W2,C-M3-W3,C-M3-W4,C-M3-W5,D-M4-W1,D-M4-W2,D-M4-W3,D-M4-W4,D-M4-W5,E-M5-W1,E-M5-W2,E-M5-W3,E-M5-W4,E-M5-W5,F-M6-W1,F-M6-W2,F-M6-W3,F-M6-W4,F-M6-W5,G-M7-W1,G-M7-W2,G-M7-W3,G-M7-W4,G-M7-W5,H-M8-W1,H-M8-W2,H-M8-W3,H-M8-W4,H-M8-W5,I-M9-W1,I-M9-W2,I-M9-W3,I-M9-W4,I-M9-W5,J-M10-W1,J-M10-W2,J-M10-W3,J-M10-W4,J-M10-W5,K-M11-W1,K-M11-W2,K-M11-W3,K-M11-W4,K-M11-W5,L-M12-W1,L-M12-W2,L-M12-W3,L-M12-W4,L-M12-W5
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,-4.111013,5.919693,4.618783,nan,-8.315871,-10.921317,-0.812466,19.629947,nan,5.454545,23.060129,4.203503,3.263264,19.761911,-20.885624,37.638400,13.291490,22.320443,nan,33.447675,26.185547,-28.420088,1.602667,nan
2018,17.748922,-16.917722,-14.453056,3.035126,-31.104003,-1.419439,28.765647,-7.566354,20.072993,-17.207382,-14.093147,3.433264,-19.564376,nan,3.008802,19.169196,5.169915,7.208304,2.569491,-10.157416,-1.759621,-13.932030,5.113374,nan,-12.305393,-4.662084,-4.765474,3.565686,nan,5.522558,-5.349890,16.377328,11.037786,-14.453799,-10.191353,2.678424,3.434488,8.985224,nan,-14.395919,4.042378,3.198730,-1.196396,nan,-0.170677,-4.170962,3.955775,-1.520306,-0.062867,-0.555671,-12.226716,-27.837808,1.768782,nan,-14.786026,-9.012848,21.692235,-3.252148,4.850754
2019,-12.809897,1.774808,0.327187,-2.589823,nan,6.418246,-0.330729,2.070553,1.714678,2.876039,1.663692,0.259427,2.821101,nan,26.029144,-0.753732,2.433311,0.953131,8.772630,20.684094,16.937419,5.725251,1.317149,nan,-12.581595,17.381158,21.814699,-0.475429,nan,5.086926,-10.831025,4.081607,-9.901748,14.543669,5.672283,-10.679361,-1.586234,-4.118335,nan,6.762831,-0.763204,-2.663893,-19.786798,-2.347247,5.343017,-0.627051,15.888659,-3.500335,nan,-1.707070,-5.948567,-18.812354,7.110777,nan,1.604681,-5.213239,5.363223,-1.499458,-0.401709
2020,11.244638,6.313157,-0.996585,8.324675,nan,8.789070,-2.309461,0.259316,-14.135099,-5.826173,-33.273676,8.504327,1.123959,15.173540,1.934361,3.142477,8.027916,15.612172,nan,-1.926885,10.974381,-9.924172,8.374837,nan,3.161637,-4.154000,-0.507488,-1.918730,-0.512826,2.572824,-1.013833,7.846137,11.476669,nan,5.512602,1.962987,-2.206869,0.541203,-12.423780,0.750203,5.685175,-1.337237,-0.998955,nan,6.585029,1.179697,13.263248,5.616886,nan,12.452213,3.114035,15.400326,-1.243030,6.458070,-0.948826,22.323506,12.048838,25.563609,nan
2021,15.605946,-6.084951,-9.951405,2.583832,nan,17.232421,25.214399,18.172117,-21.383361,12.933987,15.716915,-2.754540,-2.744354,4.346472,3.093398,-6.414778,-12.614851,15.308633,nan,2.938640,-20.276720,-25.355123,2.767561,0.435030,9.007241,-8.765635,-2.527600,1.682934,nan,-2.916391,-7.240258,11.336099,12.618121,nan,9.897298,7.259976,4.822686,-0.957347,6.129121,-11.074161,2.643159,-8.638228,11.675174,nan,13.400391,12.567589,-1.098844,0.735520,nan,3.219879,3.548906,-10.526837,-2.290810,-13.755696,1.331233,-6.738076,8.770654,-6.872018,nan
2022,-11.465422,2.883198,-15.850585,4.517121,11.876748,-0.772070,-8.719243,-1.791810,1.914450,-1.674770,9.224469,13.488525,-0.897779,nan,-9.154819,-5.884245,-0.574574,-2.488636,nan,-11.515831,-7.960130,-3.303500,-2.726058,1.530842,-11.179009,-22.580004,2.255614,-8.186346,nan,8.007111,-0.301758,8.558346,3.172192,nan,-0.523558,4.879740,-11.477520,-9.114269,2.273977,9.132713,-11.044598,-3.141865,1.316754,nan,2.000657,-0.910393,1.595911,5.401423,1.348202,-21.887601,-0.315642,0.916981,4.120394,nan,-0.126267,-2.030079,0.556246,-1.279459,nan
2023,3.073086,21.857235,8.790739,4.563900,-3.413416,-5.011880,11.428463,-2.956678,-4.774431,-1.931009,27.160680,-0.017231,0.728760,nan,0.539155,6.993740,-8.955916,5.953552,nan,-2.747219,-5.319962,-0.630925,4.924596,-3.384285,-4.387392,1.598540,15.651796,0.506719,nan,-1.490380,0.236334,-0.490374,-2.668085,-0.658001,0.740535,-10.626112,-0.336884,-0.500234,nan,-0